In [88]:
import pandas as pd

In [89]:
# global recoding on age and pram on sex 
anon_data_global_recoding_pram = pd.read_excel("anon_data/anon_data_global_recoding_pram.xlsx")
# global recoding on age, global recoding on marital status 
anon_data_global2 = pd.read_excel("anon_data/anon_data_global2.xlsx")
# global recodign on age, global recoding on marital status, and pram on sex 
anon_data_global2_pram = pd.read_excel("anon_data/anon_data_global2_pram.xlsx")

results_data = pd.read_excel("data/public_data_resultsE.xlsx")
pub_data_register = pd.read_excel('data/public_data_registerE.xlsx')

c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\obe\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


#### Testing if the anonymised datasets can be de-anonymised 

In [90]:
quasi_identifiers1 = ["age_group", "citizenship", "marital_status", "zip"]
quasi_identifiers2 = ["age_group", "citizenship", "marital_status", "zip", "sex"]

def k_anonymity_violations(data, quasi_identifiers, k_levels=[2, 3, 5]):
    # Group by quasi-identifiers to get the counts for each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Dictionary to store the number of rows violating each k-level
    violations = {}
    total_rows = len(data)

    for k in k_levels:
        # Count rows where the group size is less than k
        violating_rows = grouped[grouped['count'] < k]['count'].sum()
        # Store as a percentage for comparison with R's output
        violations[k] = violating_rows, (violating_rows / total_rows) * 100

    return violations

In [91]:
def identify_k_anonymity_violations(data, quasi_identifiers, k=2):
    # Group by quasi-identifiers to count each unique combination
    grouped = data.groupby(quasi_identifiers).size().reset_index(name='count')
    
    # Filter for combinations where the count is less than k (i.e., violate k-anonymity)
    violating_combinations = grouped[grouped['count'] < k]
    
    # Merge to find the original rows that match these violating combinations
    violations = data.merge(violating_combinations.drop(columns='count'), on=quasi_identifiers, how='inner')
    
    return violations

Global recoding on age, pram on sex

In [92]:
k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)
vioalting_rows_global1_pram = identify_k_anonymity_violations(anon_data_global_recoding_pram, quasi_identifiers1)

Global recoding on age, global recoding on marital status 

In [93]:
k_anonymity_violations(anon_data_global2, quasi_identifiers1)
vioalting_rows_global2 = identify_k_anonymity_violations(anon_data_global2, quasi_identifiers1)

In [94]:
k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)
vioalting_rows_global2_pram = identify_k_anonymity_violations(anon_data_global2_pram, quasi_identifiers1)